# Module 2 Summative Lab

## Introduction

For today's section, we're going to work on a single big lab to apply everything we've learned in Mod 2!

## About This Lab

A quick note before getting started--this lab isn't like other labs you seen so far. This lab is meant to take ~8 hours to complete, so it's much longer and more challenging than the average labs you've seen so far. If you feel like this lab is challenging or that you might be struggling a bit, don't fret--that's by design! With everything we've learned about Web Scraping, APIs, and Databases, the best way to test our knowledge of it is to build something substantial! 

## The Project

In this lab, we're going to make use of everything we've learned about APIs, databases, and Object-Oriented Programming to **_Extract, Transform, and Load_** (or **_ETL_**, for short) some data from a SQL database into a MongoDB Database. 

You'll find a database containing information about soccer teams and the matches they've played in the file `database.sqlite`. For this project, our goal is to get the data we think is important from this SQL database, do some calculations and data transformation, and then store everything in a MongoDB database. 

Let's get into the specifics of this project.

### The Goal

Start by examining the data dictionary for the SQL database we'll be working with, which comes from this [kaggle page](https://www.kaggle.com/laudanum/footballdelphi).  Familiarize yourself with the tables it contains, and what each column means. We'll be using this database to get data on each soccer team, calculate some summary statistics, and then store each in a MongoDB database. 

Upon completion of this lab, each unique team in this dataset should have a record in the MongoDB instance containing the following information:

* The name of the team
* The total number of goals scored by the team during the 2011 season
* The total number of wins the team earned during the 2011 season
* A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly by assigning it to a variable)
* The team's win percentage on days where it was raining during games in the 2011 season. 



In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import sqlite3

In [ ]:
con = sqlite3.connect('database.sqlite')
pd.read_sql("SELECT name FROM sqlite_master WHERE type ='table'",con)

In [ ]:
df = pd.read_sql("""
SELECT * 
FROM Matches
""", con, parse_dates=['Date'])
df = df[match_df.Season == 2011]
df

In [ ]:
df.query("HomeTeam=='Aachen'")[["HomeTeam","FTHG","Date","FTR"]] \
    .rename(columns={"HomeTeam":"Team","FTHG":"Goals"}) \
      .append(df.query("AwayTeam=='Aachen'")[["AwayTeam","FTAG","Date"]] \
          .rename(columns={"AwayTeam":"Team","FTAG":"Goals"}))

In [ ]:
def ftr_to_winloss(ftr, is_home):
    if ftr == 'D':
        return ftr
    if (is_home and (ftr =='H')) or (not is_home and (ftr == 'A')):
        return 'W'
    return 'L'

df.query("HomeTeam=='Aachen'")[["HomeTeam",
                                "FTHG",
                                "Date",
                                "FTR"]].apply(lambda col: 
                                                col.apply(lambda row: 
                                                            ftr_to_winloss(row, True)) if col.name == 'FTR' else col)

In [ ]:
tempdf = df.query("HomeTeam=='Aachen'").copy()
tempdf['is_home'] = 1
tempdf = tempdf.drop(columns='Season')
tempdf.FTR = tempdf.FTR.apply(ftr_to_winloss, args=[True])
tempdf = tempdf.rename(columns=lambda x: x.lower())
tempdf.rename(columns={'hometeam':'team','awayteam':'opponent','fthg':'goals','ftag':'opp_goals'})

In [ ]:
df.query("HomeTeam=='Aachen'")[["HomeTeam","FTHG","Date","FTR"]].rename(columns={'HomeTeam':'team',1:'goals',2:'date',3:'ftr'})

In [ ]:
list(np.unique(df[["HomeTeam", "AwayTeam"]].values.ravel('F')))

In [ ]:
# total goals scored
for team in np.unique(df[["HomeTeam", "AwayTeam"]].values.ravel('F')):
    print(team, 
          pd.Series(df.query("HomeTeam=='{}'".format(team.replace("'","\\'")))["FTHG"]) \
          .append(pd.Series(df.query("AwayTeam=='{}'".format(team.replace("'","\\'")))["FTAG"])).sum())

In [ ]:
def game_goals(df, team, home_or_away):
    """returns pd.Series with goals scored by team; home_or_away is 'H' or 'A'"""
    if home_or_away == 'H':
        teamcol = 'HomeTeam'   
    elif home_or_away == 'A':
        teamcol = 'AwayTeam'
    else:
        raise ValueError(f"Invalid value {home_or_away}. Must be 'H' or 'A'.")
    gamecol = 'FT' + home_or_away + 'G'
    team_esc = team.replace("'","\\'")
    return pd.Series(df.query(f"{teamcol}=='{team_esc}'")[gamecol])

def total_goals(df, team):
    """returns total goals scored by team"""
    return game_goals(df, team, 'H').append(game_goals(df, team, 'A')).sum()

total_goals(df, 'Liverpool')

In [ ]:
# total wins
for team in np.unique(df[["HomeTeam", "AwayTeam"]].values.ravel('F')):
    print(team, df.query("(HomeTeam=='{team}' and FTR=='H') or (AwayTeam=='{team}' and FTR=='A')" \
                   .format(team=team.replace("'","\\'"))).shape[0])

# Parser

In [ ]:
import soccerparser
from soccerparser import SoccerParser
import importlib
importlib.reload(soccerparser)



In [ ]:
sp = SoccerParser('database.sqlite')
sp.parse()
sp.results_df.head()

In [ ]:
sp.match_df

In [ ]:
sp.match_df.query("team=='Swansea'").head()

_A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly by assigning it to a variable)_

Correct this to a win-loss bar chart for each team

In [ ]:
importlib.reload(plt)
sns.set(style="darkgrid")
team = 'Tottenham'
sns.countplot(data=sp.match_df.query(f"team == '{team}'"), x='result', order=['W', 'L', 'D'], palette=sns.xkcd_palette(['emerald', 'cherry', 'grey']))
plt.title(f"{team} 2011 Season", fontsize='x-large')

In [ ]:
fig, ax = sp.team_win_chart('Arsenal')
plt.show()

In [ ]:
sp.match_df.query('match_id == 1138')

In [ ]:
sp.match_df.head()

In [ ]:
# You don't have to use these classes, but we recommend them as a good place to start!
class WeatherGetter():
    pass

In [ ]:
class MongoHandler():
    pass

#### Getting the Weather Data

Note that for this last calculation, you'll need to figure out if it was raining or not during the game. The database itself does not contain this information, but it does contain the date on which the game was played. For this, you'll need to use the [DarkSky API](https://darksky.net/dev) to get the historical weather data for that day. Note that each game is played in a different location, and this information is not contained in our SQL database. However, the teams in this database are largely german, so go ahead and just use the weather in Berlin, Germany as a proxy for this information. If it was raining in Berlin on the day the game was played, count that as rain game--**_you do not need to try and figure out the actual weather at each game's location, because we don't have that information!_**

#### NOTE: The DarkSky API is limited to 1000 free API calls a day, so be sure to test your model on very small samples. Otherwise, you'll hit the rate limit!



## Get team locations

In [ ]:
germany_teams_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_football_clubs_in_Germany')[0]

In [ ]:
my_teams_df = pd.DataFrame(columns=['team', 'city', 'country'])

for team in np.unique(df[["HomeTeam", "AwayTeam"]].values.ravel('F')):
    if not germany_teams_df[germany_teams_df.Club.str.contains(team)]['City'].empty:
        city = germany_teams_df[germany_teams_df.Club.str.contains(team)]['City'].values[0]
        my_teams_df = my_teams_df.append({'team' : team,
                            'city' : city,
                            'country' : 'Germany'}, ignore_index = True)

my_teams_df

In [ ]:
england_teams_df = pd.read_html('https://en.wikipedia.org/wiki/2011%E2%80%9312_Premier_League')[1]

for team in np.unique(df[["HomeTeam", "AwayTeam"]].values.ravel('F')):
    if not england_teams_df[england_teams_df.Team.str.contains(team)]['Location'].empty:
        city = england_teams_df[england_teams_df.Team.str.contains(team)]['Location'].values[0]
        my_teams_df = my_teams_df.append({'team' : team,
                            'city' : city,
                            'country' : 'England'}, ignore_index = True)

In [ ]:
my_teams_df

In [ ]:
from geopy.geocoders import Nominatim

In [ ]:
geolocator = Nominatim(user_agent="soccer_weather")

def geolocate(city, country):
    city = re.sub(' *\([^)]*\)', '', city)
    return geolocator.geocode(f"{city}, {country}")
    
from geopy.extra.rate_limiter import RateLimiter
# geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
my_teams_df['location'] = my_teams_df.apply(lambda x: geolocate(x.city, x.country), axis=1)

In [ ]:
my_teams_df['point'] = my_teams_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [ ]:
my_teams_df

In [ ]:
my_teams_df.to_csv('team_locations.csv')

In [ ]:
pd.read_csv('team_locations.csv').query("city == 'Berlin'")['point'].drop_duplicates()

## Project Architecture

Unlike previous labs, this lab is more open-ended, and will require you to make design decisions and plan out your strategy for building a system with this many working parts. However, **_using Object-Oriented Programming is a requirement for this project--you must create at least 2 separate, well-structured classes in your solution!_** Although it may seem easier to "just start coding", this is a classic beginner's mistake. Instead, think about separating out the different functionalities you'll need to reach your goal, and then build classes to handle each. For instance, at a minimum, you'll need to:

* Query the SQL database
* Calculate summary statistics
* Get the weather data from the DarkSky API
* Load the data into MongoDB

We **_strongly recommend_** you consider creating separate classes for handling at least some of these tasks.  Be sure to plan the inputs, outputs, and methods for each class before you begin coding! 

**_NOTE:_** We have provided some empty classes below. You are welcome to delete them and use a different architecture for this project if you so choose.  You do not have to use each of them, they are just there to give you an idea of what sorts of classes you may want to consider using.

### Rapid Prototyping and Refactoring

It's totally okay to try to get a task working without using OOP. For instance, when experimenting with the DarkSky API for getting historical weather data, it makes sense to just write the code in the cells and rapidly iterate until you get it all working. However, once you get it working, you're not done--you should then **_Refactor_** your code into functions or classes to make your code more modular, reusable, understandable, and maintainable! 

In short--do what you need to do to get each separate piece of functionality working, and then refactor it into a class after you've figured it out!

### Some Final Advice

You haven't built anything this big or complex thus far, so you may not yet fully realize how much trial and error goes into it. If your code keeps breaking, resist the urge to get frustrated, and just keep working. Software development is an iterative process!  No one writes perfect code that works the first time for something this involved. You're going to run into _a lot_ of small errors in this project, right up until the point where it just works, and then you're done! However, you can reduce these errors by planning out your code, and thinking about how all of the pieces fit together before you begin coding. Once you have some basic understanding of how it all will work, then you'll know what you need to build, and then all that is left is to build it!

In short:

* Plan ahead--you'll thank yourself later!
* Errors and broken code aren't bad, they're normal. 
* Keep working, and stay confident--you can do this!

Good luck--we look forward to seeing your completed project!

# Summary

In this lab, we dug deep and used everything we've learned so far about python programming, databases, HTTP requests and API calls to ETL data from a SQL database into a MongoDB instance!